In [1]:
import arrow
import pandas as pd
de_train = pd.read_parquet("/home/roger/Github/OpenProblems/de_train.parquet")
id_map = pd.read_csv("/home/roger/Github/OpenProblems/id_map.csv")
sample_submission = pd.read_csv("/home/roger/Github/OpenProblems/sample_submission.csv")
from sklearn.linear_model import Ridge
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
import category_encoders as ce
# Create features and and labels for reverse model 18211 features and 152 labels for true model
features_columns = ["cell_type", "sm_name"]
#Característiques, son les dues variables categòriques
labels_columns=["cell_type","sm_name","sm_lincs_id","SMILES","control"]
#Etiquetes
#Generem la columna que volem com a resposta "labels", 
labels = de_train.drop(columns=labels_columns)
#i la que ens servirà per fer la predicció, "features"
features = pd.DataFrame(de_train, columns=features_columns)
features
test_data = pd.DataFrame(id_map, columns=features_columns)
from sklearn.preprocessing import OneHotEncoder

# Create an instance of the encoder
encoder = OneHotEncoder()

# Fit the encoder on features
encoder.fit(features)

# Transform the features into one-hot encoded format
one_hot_encode_features = encoder.transform(features)

# Transform the test data(id_map)
one_hot_test = encoder.transform(test_data)
one_hot_encode_features.toarray().shape, one_hot_test.toarray().shape


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


((614, 152), (255, 152))

In [2]:
from sklearn.model_selection import train_test_split

# Split the data into 70% training, 15% validation, and 15% testing
X_train, X_temp, y_train, y_temp = train_test_split(one_hot_encode_features, labels.values, test_size=0.3, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)

#El de validació, ens serveix per ajustar hiperparàmetres i evaluar el rendiment durant les prediccions

In [58]:
y_val

array([[-0.33047679,  0.17437263,  0.0841885 , ...,  0.04805798,
         0.95751343,  0.04815479],
       [-0.1625888 ,  0.37516091, -0.03813352, ..., -0.70248078,
        -0.2144884 , -0.38521405],
       [ 1.02297612,  0.23691605, -0.05082962, ..., -0.95580662,
        -0.15872175, -0.76777753],
       ...,
       [ 0.19343401,  0.18700177,  1.94101059, ...,  0.63578256,
         0.30091215,  0.23285415],
       [-1.58172233, -0.33120683, -0.00388089, ...,  0.05873893,
         0.43647877,  1.7079427 ],
       [ 0.31744741,  0.09886952, -0.29193862, ..., -0.03029502,
        -0.16007748, -0.5346985 ]])

In [3]:
# We also get full features for final training 
full_features = one_hot_encode_features.toarray()
full_labels = labels.values

In [4]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Crear el modelo de Regresión Ridge
ridge_model = Ridge(alpha=1.0)  # Puedes ajustar el valor de alpha según tus necesidades

# Entrenar el modelo en los datos de entrenamiento
ridge_model.fit(X_train, y_train)

# Realizar predicciones en los datos de validación
y_val_pred_ridge = ridge_model.predict(X_val)

# Calcular el error cuadrático medio (MSE) en los datos de validación
mse_ridge = mean_squared_error(y_val, y_val_pred_ridge)
print(f'Error cuadrático medio en datos de validación (Ridge): {mse_ridge:.2f}')

Error cuadrático medio en datos de validación (Ridge): 1.07


In [11]:
ridge_model.fit(full_features, full_labels)

Ridge()

In [12]:
preds= ridge_model.predict(one_hot_test.toarray())

In [32]:
preds

array([[ 0.83606719, -0.18691551,  1.26961024, ...,  0.56216068,
        -0.09362092,  0.53028876],
       [ 0.53082526, -0.09111874,  1.00408857, ...,  0.07083571,
         0.13674159,  0.01418158],
       [ 0.92226332,  0.16108907,  1.02505399, ...,  0.16052588,
        -0.02650789, -0.49365602],
       ...,
       [ 0.69691978,  0.53411328, -3.29669123, ..., -0.52346247,
         0.26159229, -0.35825097],
       [ 0.83494149,  0.12433736, -2.66210155, ..., -0.50698495,
        -0.0224148 ,  0.16639476],
       [ 0.51336656,  0.25925395, -3.03082561, ..., -0.61281555,
        -0.4212486 , -0.39069892]])

In [5]:
cols_public = [0, 2, 3, 4, 7, 14, 16, 17, 18, 22, 26, 36, 43, 45, 47, 48, 51, 52, 55, 58, 60, 68, 69, 70, 71, 74, 78, 79, 81, 82, 85, 86, 88, 90, 91, 95, 97, 98, 103, 105, 106, 108, 109, 111, 113, 115, 117, 118, 119, 122, 128, 130, 131, 132, 135, 142, 144, 145, 146, 149, 154, 163, 170, 172, 174, 175, 178, 179, 182, 185, 187, 195, 196, 197, 198, 201, 205, 206, 208, 209, 212, 213, 215, 217, 218, 222, 224, 225, 230, 232, 233, 235, 236, 238, 240, 242, 244, 245, 246, 249]


In [6]:
provem = pd.read_csv("/home/roger/Baixades/entrega.csv")

In [7]:
provem

,id,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0,0.462067,0.333164,0.282393,0.864314,1.103537,0.681204,-0.011400,0.511585,0.091167,...,-0.487438,-0.076190,-0.074838,0.360439,0.399049,0.653428,0.619294,0.534062,-0.658728,0.041275
1,1,-0.077632,0.014408,0.065511,0.031914,0.278137,0.326295,-0.030011,0.139525,0.150199,...,0.013403,0.030177,-0.047023,0.109163,0.123197,0.029427,0.115830,0.036995,-0.050327,-0.047864
2,2,4.516325,4.453264,4.074874,4.208816,4.796946,5.500780,4.201675,4.470567,4.194056,...,4.245542,4.141732,4.264077,4.467162,4.335662,4.475067,4.417105,4.258112,4.102206,4.052730
3,3,0.061848,0.264810,0.280930,0.296526,0.416424,0.352968,0.115435,0.344529,0.244837,...,0.095571,0.175771,0.151149,0.319440,0.310137,0.279016,0.276906,0.243324,0.156719,0.114914
4,4,0.108198,0.015672,0.119478,0.222975,0.460281,0.378904,-0.039368,0.199365,-0.022555,...,-0.268710,-0.055975,-0.104511,0.077440,0.129164,0.191859,0.162400,0.181690,-0.155262,-0.069485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,250,0.078685,0.047404,-0.134128,-0.020459,0.282464,-0.153848,-0.008667,0.120142,0.155904,...,-0.185034,-0.209786,-0.009008,-0.028455,0.071896,-0.064984,0.035646,0.131742,-0.072857,-0.014499
251,251,0.192609,0.015485,0.267063,0.230512,1.300028,0.551453,0.099151,-0.022518,0.393088,...,-0.446241,-0.508816,0.125177,-0.094444,0.305428,0.084690,-0.031948,0.223898,0.033906,-0.009356
252,252,0.053263,0.148030,-0.022974,0.046427,0.364049,-0.052244,-0.044395,0.071997,0.107999,...,-0.241901,-0.188191,0.083745,-0.043139,0.120209,-0.040359,0.019710,0.183417,-0.014529,0.018427
253,253,1.072385,0.484715,-0.285435,0.361732,3.906288,2.971852,0.429157,0.514162,0.396571,...,-0.488719,0.278769,-0.455760,-0.515792,1.173396,0.572009,0.002834,0.209050,-0.308653,0.263124


In [13]:
sample_columns = sample_submission.columns
sample_columns= sample_columns[1:]
submission_df = pd.DataFrame(preds, columns=sample_columns)

submission_df.insert(0, 'id', range(255))
submission_df

,id,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0,0.836067,-0.186916,1.269610,2.588464,4.479347,2.045678,0.045861,0.082917,-0.026778,...,-0.743166,-0.401537,0.204744,0.938165,0.553710,0.462624,0.601329,0.562161,-0.093621,0.530289
1,1,0.530825,-0.091119,1.004089,1.312936,4.068333,1.674339,-0.182880,-0.066687,0.928607,...,0.217125,0.226355,-0.109065,0.617210,0.282545,-0.260431,0.336318,0.070836,0.136742,0.014182
2,2,0.922263,0.161089,1.025054,1.374218,4.230585,2.388926,0.547062,0.065757,0.577713,...,0.021730,0.006438,-0.015602,1.060429,0.243070,0.054394,0.450941,0.160526,-0.026508,-0.493656
3,3,0.570949,0.329949,1.038886,1.606745,3.878553,1.499426,-0.193598,0.559794,0.658955,...,-0.208176,0.202735,0.116262,0.978268,0.124533,-0.092142,0.245998,0.379811,0.020010,0.221836
4,4,0.790043,-0.317642,1.065745,1.627450,3.890081,1.614551,-0.237058,0.044030,0.481943,...,-0.691252,0.079437,-0.134584,0.459632,0.150476,0.165591,0.119920,0.714333,0.100187,0.152138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,250,0.769914,-0.425520,-3.209610,0.351987,5.615472,1.442813,0.361209,-0.095014,0.072917,...,-0.535507,-0.630063,-1.779295,-0.865807,0.825871,-0.428725,-0.822883,-0.680739,0.109591,0.153240
251,251,0.925176,-0.255033,-2.939405,1.100448,5.758857,2.057700,0.183926,-0.425187,0.057853,...,-0.607785,-0.676497,-1.360051,-0.653809,1.164733,-0.254677,-1.060461,-0.390140,-0.196227,-0.227099
252,252,0.696920,0.534113,-3.296691,0.621068,5.545198,1.650619,0.163571,-0.577431,0.215439,...,-0.591123,-0.544190,-1.055662,-0.656498,1.052167,-0.251400,-1.031428,-0.523462,0.261592,-0.358251
253,253,0.834941,0.124337,-2.662102,0.738762,5.801982,1.684676,0.750487,0.457592,1.572173,...,-0.369705,-0.150883,-1.800492,-0.870309,0.519432,0.127986,-0.993186,-0.506985,-0.022415,0.166395


In [14]:
public_data_rows = submission_df[submission_df['id'].isin(cols_public)]
provem_public_rows = provem[provem['id'].isin(cols_public)]

In [34]:
summary_public = public_data_rows.describe()
summary_other = provem_public_rows.describe()

differences = summary_public - summary_other
print(differences)

        id      A1BG  A1BG-AS1       A2M   A2M-AS1      A2MP1     A4GALT  \
count  0.0  0.000000  0.000000  0.000000  0.000000   0.000000   0.000000   
mean   0.0  0.411952 -0.165840 -1.865120  0.671690   3.097756   0.669488   
std    0.0 -0.837653 -0.403920  1.585488 -0.446263  -1.432712  -1.686491   
min    0.0  0.183406 -0.081393 -2.583044  1.292546   2.956841   0.871751   
25%    0.0  0.691266 -0.154886 -4.049408  0.616291   3.277627   1.208911   
50%    0.0  0.776649 -0.065827 -1.763855  0.787257   4.494376   1.316044   
75%    0.0  0.551565  0.280506  0.598377  1.006649   4.336917   0.991306   
max    0.0 -7.471971 -4.483419 -5.157558 -4.324324 -11.623680 -17.988017   

           AAAS      AACS     AAGAB  ...      ZUP1      ZW10    ZWILCH  \
count  0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000   
mean  -0.023456 -0.198173  0.260354  ... -0.029499 -0.225181 -1.116140   
std   -0.314608 -0.400713 -0.258497  ... -0.175015 -0.322539  0.150129   
min    0.135658 -0.

In [15]:
no_public_data_rows = submission_df[~submission_df['id'].isin(cols_public)]
no_provem_public_rows = provem[~provem['id'].isin(cols_public)]

In [36]:
no_summary_public = no_public_data_rows.describe()
no_summary_other = no_provem_public_rows.describe()

no_differences = no_summary_public - no_summary_other
print(no_differences)

        id      A1BG  A1BG-AS1       A2M   A2M-AS1      A2MP1    A4GALT  \
count  0.0  0.000000  0.000000  0.000000  0.000000   0.000000  0.000000   
mean   0.0  0.589976 -0.031875 -1.809986  0.806611   3.368697  0.873194   
std    0.0 -0.274120  0.165549  2.068633 -0.020111  -0.895676 -0.718172   
min    0.0  0.390000 -0.799634 -4.300715  0.496659   2.901127  0.695402   
25%    0.0  0.630158 -0.147062 -4.135375  0.597690   3.211926  1.084797   
50%    0.0  0.771815  0.001721 -0.001745  0.884948   4.483168  1.285438   
75%    0.0  0.669751  0.311371  0.613025  1.053966   4.508917  1.077352   
max    0.0 -2.414826  0.000687 -2.787060 -2.750057 -11.139484 -6.449137   

           AAAS      AACS     AAGAB  ...      ZUP1      ZW10    ZWILCH  \
count  0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000   
mean   0.086268 -0.177468  0.361438  ...  0.177466 -0.125851 -1.041558   
std    0.166303  0.070682  0.165747  ...  0.162822  0.135214  0.689912   
min   -0.427943 -0.958243 -0

In [37]:
mse_ridge = mean_squared_error(public_data_rows, provem_public_rows)
print(f'Error cuadrático medio en datos de validación (Ridge): {mse_ridge:.2f}')

Error cuadrático medio en datos de validación (Ridge): 2.88


In [38]:
mse_ridge = mean_squared_error(no_public_data_rows, no_provem_public_rows)
print(f'Error cuadrático medio en datos de validación (Ridge): {mse_ridge:.2f}')

Error cuadrático medio en datos de validación (Ridge): 1.77


### JUGUEM AMB LES DADES QUE TENIM!

Hauriem de crear els nostres X_val i y_val

In [16]:
df = provem_public_rows.drop('id', axis=1)
df

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0.462067,0.333164,0.282393,0.864314,1.103537,0.681204,-0.011400,0.511585,0.091167,0.768242,...,-0.487438,-0.076190,-0.074838,0.360439,0.399049,0.653428,0.619294,0.534062,-0.658728,0.041275
2,4.516325,4.453264,4.074874,4.208816,4.796946,5.500780,4.201675,4.470567,4.194056,4.267981,...,4.245542,4.141732,4.264077,4.467162,4.335662,4.475067,4.417105,4.258112,4.102206,4.052730
3,0.061848,0.264810,0.280930,0.296526,0.416424,0.352968,0.115435,0.344529,0.244837,0.306557,...,0.095571,0.175771,0.151149,0.319440,0.310137,0.279016,0.276906,0.243324,0.156719,0.114914
4,0.108198,0.015672,0.119478,0.222975,0.460281,0.378904,-0.039368,0.199365,-0.022555,0.297477,...,-0.268710,-0.055975,-0.104511,0.077440,0.129164,0.191859,0.162400,0.181690,-0.155262,-0.069485
7,0.017116,-0.023381,0.214604,0.437055,0.540571,0.375075,-0.114287,0.291052,-0.052531,0.370685,...,-0.413995,-0.167151,-0.275228,0.108914,0.111943,0.267859,0.222184,0.237452,-0.330703,-0.109349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,1.247309,0.578961,0.551067,1.130912,5.200572,2.784500,0.707802,0.406606,0.555574,0.760566,...,-0.868539,-0.169519,0.305517,-0.631862,1.542295,0.729348,0.456026,0.707201,-0.044002,0.765652
244,0.181579,0.144406,0.233969,0.098897,0.635072,0.227591,0.180627,0.165571,0.220445,0.257921,...,-0.026890,-0.103319,0.137398,-0.024586,0.221549,0.115677,0.009149,0.103917,0.062139,0.091301
245,0.096385,0.093463,0.128921,0.107463,0.558667,0.091153,-0.036066,0.015429,0.108094,0.180066,...,-0.261751,-0.269424,0.055420,-0.174134,0.094984,0.000811,0.005686,0.149935,-0.050900,0.019170
246,0.024928,0.068197,-0.146074,-0.041722,0.451191,0.048453,-0.042733,0.084725,0.139246,0.164528,...,-0.284240,-0.155504,0.040756,-0.045641,0.113998,-0.036030,0.071052,0.067279,-0.112358,0.002270


In [17]:
y_val2 = df.values
y_val2

array([[ 4.62067441e-01,  3.33164236e-01,  2.82392933e-01, ...,
         5.34061797e-01, -6.58727817e-01,  4.12750760e-02],
       [ 4.51632463e+00,  4.45326366e+00,  4.07487375e+00, ...,
         4.25811234e+00,  4.10220566e+00,  4.05273027e+00],
       [ 6.18484747e-02,  2.64809729e-01,  2.80929539e-01, ...,
         2.43323658e-01,  1.56719068e-01,  1.14914329e-01],
       ...,
       [ 9.63851946e-02,  9.34631753e-02,  1.28920964e-01, ...,
         1.49934601e-01, -5.09000575e-02,  1.91701466e-02],
       [ 2.49277018e-02,  6.81973377e-02, -1.46074217e-01, ...,
         6.72794369e-02, -1.12358336e-01,  2.27039471e-03],
       [ 4.86877577e+00,  2.96296283e+00, -7.00295968e-01, ...,
         8.03092207e-01,  8.56763141e-01,  1.23568862e+00]])

In [18]:
test_data2 = id_map[id_map['id'].isin(cols_public)]
test_data2 = test_data2.drop('id', axis=1)
one_hot_test2 = encoder.transform(test_data2)
one_hot_test2

<100x152 sparse matrix of type '<class 'numpy.float64'>'
	with 200 stored elements in Compressed Sparse Row format>

In [19]:
import numpy as np
rows_to_keep = np.setdiff1d(np.arange(one_hot_test2.shape[0]), cols_public)

In [20]:
one_hot_test_sin_filas = one_hot_test2[rows_to_keep]

In [21]:
y_val_pred_ridge = ridge_model.predict(one_hot_test2)

# Calcular el error cuadrático medio (MSE) en los datos de validación
mse_ridge = mean_squared_error(y_val2, y_val_pred_ridge)
mse_ridge

2.7701129947695193

Optimal alpha: 1.0
Mean Cross-Validated MSE on Validation Set: 2.50


In [28]:
%%time
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

# Crear el modelo de Regresión de Bosques Aleatorios
rf_model = RandomForestRegressor(n_estimators=300, random_state=42)

# Definir los hiperparámetros que deseas probar
param_grid = {
    'n_estimators': [300],  # Puedes ajustar el número de árboles
    'max_depth': [10],  # Puedes ajustar la profundidad máxima del árbol
    'min_samples_split': [5],  # Puedes ajustar el número mínimo de muestras para dividir un nodo
    'min_samples_leaf': [2]  # Puedes ajustar el número mínimo de muestras en una hoja
}

# Crear un objeto GridSearchCV
grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='neg_mean_squared_error')

# Realizar la búsqueda de hiperparámetros en los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros
best_params = grid_search.best_params_
print("Mejores hiperparámetros:", best_params)

# Obtener el modelo con los mejores hiperparámetros
best_rf_model = grid_search.best_estimator_

# Realizar predicciones en los datos de validación
y_val_pred_rf = best_rf_model.predict(one_hot_test2)

# Calcular el error cuadrático medio (MSE) en los datos de validación
mse_rf = mean_squared_error(y_val2, y_val_pred_rf)
print(f'Error cuadrático medio en datos de validación (Random Forest): {mse_rf:.2f}')


Mejores hiperparámetros: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 300}
Error cuadrático medio en datos de validación (Random Forest): 1.81
CPU times: user 6min 30s, sys: 1.97 s, total: 6min 32s
Wall time: 6min 36s


In [114]:
rf_model.fit(full_features, full_labels)

RandomForestRegressor(n_estimators=300, random_state=42)

In [115]:
preds2= rf_model.predict(one_hot_test.toarray())

In [116]:
sample_columns = sample_submission.columns
sample_columns= sample_columns[1:]
submission_df2 = pd.DataFrame(preds2, columns=sample_columns)
submission_df2.insert(0, 'id', range(255))

In [117]:
no_public_data_rows2 = submission_df2[~submission_df2['id'].isin(cols_public)]

In [119]:
mse_ridge = mean_squared_error(no_public_data_rows2, no_provem_public_rows)
print(f'Error cuadrático medio en datos de validación (Ridge): {mse_ridge:.2f}')

Error cuadrático medio en datos de validación (Ridge): 0.39


In [120]:
public_data_rows2 = submission_df2[submission_df2['id'].isin(cols_public)]


In [121]:
mse_ridge = mean_squared_error(public_data_rows2, provem_public_rows)
print(f'Error cuadrático medio en datos de validación (Ridge): {mse_ridge:.2f}')

Error cuadrático medio en datos de validación (Ridge): 1.92


In [124]:
no_summary_public2 = no_public_data_rows2.describe()
no_summary_other = no_provem_public_rows.describe()

no_differences = no_summary_public2 - no_summary_other
print(no_differences)

        id      A1BG  A1BG-AS1       A2M   A2M-AS1      A2MP1    A4GALT  \
count  0.0  0.000000  0.000000  0.000000  0.000000   0.000000  0.000000   
mean   0.0  0.067464 -0.085757  0.060064 -0.331811  -0.610189 -0.470957   
std    0.0 -0.140114  0.138662 -0.021536 -0.020830  -1.494239 -0.350024   
min    0.0 -0.093040 -0.294284 -0.648695 -0.818472  -0.298784 -0.090619   
25%    0.0  0.046268 -0.159948  0.061654 -0.463851  -0.013209 -0.078286   
50%    0.0  0.206490 -0.124950  0.104311 -0.383367   0.042705 -0.269720   
75%    0.0  0.175396 -0.034694  0.079663 -0.144147  -0.342577 -0.615457   
max    0.0 -1.365024  0.953349 -0.458985 -1.631278 -13.866607 -2.941837   

           AAAS      AACS     AAGAB  ...      ZUP1      ZW10    ZWILCH  \
count  0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000   
mean   0.091500  0.102756  0.143468  ... -0.180583  0.039195  0.013011   
std    0.134453  0.117679  0.171473  ...  0.146087  0.066909  0.181832   
min   -0.331173 -0.700242  0

In [125]:
summary_public2 = public_data_rows2.describe()
summary_other = provem_public_rows.describe()

differences = summary_public2 - summary_other
print(differences)

        id      A1BG  A1BG-AS1       A2M   A2M-AS1      A2MP1     A4GALT  \
count  0.0  0.000000  0.000000  0.000000  0.000000   0.000000   0.000000   
mean   0.0 -0.124380 -0.220182  0.028866 -0.495350  -0.895870  -0.769209   
std    0.0 -0.827702 -0.467747 -0.667795 -0.545878  -2.030326  -1.765033   
min    0.0 -0.203197 -0.206621  1.604289  0.410048  -0.458706   0.146344   
25%    0.0  0.057888 -0.153685  0.152933 -0.439993  -0.049118  -0.041498   
50%    0.0  0.223431 -0.057896  0.141320 -0.496869  -0.025606  -0.347410   
75%    0.0  0.051051 -0.069174  0.071577 -0.177367  -0.620701  -1.041810   
max    0.0 -7.903144 -4.138859 -5.137131 -5.753834 -15.689972 -20.334718   

           AAAS      AACS     AAGAB  ...      ZUP1      ZW10    ZWILCH  \
count  0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000   
mean   0.005988 -0.001856  0.024549  ... -0.320374 -0.072430 -0.085941   
std   -0.402632 -0.383237 -0.316440  ... -0.188223 -0.427489 -0.397478   
min    0.601997  0.

In [22]:
features2 = id_map[id_map['id'].isin(cols_public)]
features2 = features2.drop(['id'], axis=1)
features2 = pd.concat([features, features2], ignore_index=True)
features2

,cell_type,sm_name
0,NK cells,Clotrimazole
1,T cells CD4+,Clotrimazole
2,T cells CD8+,Clotrimazole
3,T regulatory cells,Clotrimazole
4,NK cells,Mometasone Furoate
...,...,...
709,Myeloid cells,Tamatinib
710,Myeloid cells,Tivantinib
711,Myeloid cells,Tivozanib
712,Myeloid cells,Topotecan


In [23]:
result = pd.concat([labels, provem_public_rows], ignore_index=True)
result

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,id
0,0.104720,-0.077524,-1.625596,-0.144545,0.143555,0.073229,-0.016823,0.101717,-0.005153,1.043629,...,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755,NaN
1,0.915953,-0.884380,0.371834,-0.081677,-0.498266,0.203559,0.604656,0.498592,-0.317184,0.375550,...,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887,NaN
2,-0.387721,-0.305378,0.567777,0.303895,-0.022653,-0.480681,0.467144,-0.293205,-0.005098,0.214918,...,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365,NaN
3,0.232893,0.129029,0.336897,0.486946,0.767661,0.718590,-0.162145,0.157206,-3.654218,-0.212402,...,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024,NaN
4,4.290652,-0.063864,-0.017443,-0.541154,0.570982,2.022829,0.600011,1.231275,0.236739,0.338703,...,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709,1.247309,0.578961,0.551067,1.130912,5.200572,2.784500,0.707802,0.406606,0.555574,0.760566,...,-0.169519,0.305517,-0.631862,1.542295,0.729348,0.456026,0.707201,-0.044002,0.765652,242.0
710,0.181579,0.144406,0.233969,0.098897,0.635072,0.227591,0.180627,0.165571,0.220445,0.257921,...,-0.103319,0.137398,-0.024586,0.221549,0.115677,0.009149,0.103917,0.062139,0.091301,244.0
711,0.096385,0.093463,0.128921,0.107463,0.558667,0.091153,-0.036066,0.015429,0.108094,0.180066,...,-0.269424,0.055420,-0.174134,0.094984,0.000811,0.005686,0.149935,-0.050900,0.019170,245.0
712,0.024928,0.068197,-0.146074,-0.041722,0.451191,0.048453,-0.042733,0.084725,0.139246,0.164528,...,-0.155504,0.040756,-0.045641,0.113998,-0.036030,0.071052,0.067279,-0.112358,0.002270,246.0


In [136]:
print(features)
print(features2)

              cell_type             sm_name
0              NK cells        Clotrimazole
1          T cells CD4+        Clotrimazole
2          T cells CD8+        Clotrimazole
3    T regulatory cells        Clotrimazole
4              NK cells  Mometasone Furoate
..                  ...                 ...
609  T regulatory cells        Atorvastatin
610            NK cells           Riociguat
611        T cells CD4+           Riociguat
612        T cells CD8+           Riociguat
613  T regulatory cells           Riociguat

[614 rows x 2 columns]
         cell_type                                            sm_name
0          B cells  5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...
2          B cells                                             ABT737
3          B cells                            AMD-070 (hydrochloride)
4          B cells                                            AT 7867
7          B cells                                              AZ628
..             ...          

In [24]:
# Create an instance of the encoder
encoder = OneHotEncoder()

# Fit the encoder on features
encoder.fit(features2)

# Transform the features into one-hot encoded format
one_hot_encode_features2 = encoder.transform(features2)

# Transform the test data(id_map)
one_hot_test = encoder.transform(test_data)
one_hot_encode_features2.toarray().shape, one_hot_test.toarray().shape

((714, 152), (255, 152))

In [31]:
full_features2 = one_hot_encode_features2.toarray()
full_labels2 = result.values

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0.104720,-0.077524,-1.625596,-0.144545,0.143555,0.073229,-0.016823,0.101717,-0.005153,1.043629,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,0.915953,-0.884380,0.371834,-0.081677,-0.498266,0.203559,0.604656,0.498592,-0.317184,0.375550,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,-0.387721,-0.305378,0.567777,0.303895,-0.022653,-0.480681,0.467144,-0.293205,-0.005098,0.214918,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,0.232893,0.129029,0.336897,0.486946,0.767661,0.718590,-0.162145,0.157206,-3.654218,-0.212402,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,4.290652,-0.063864,-0.017443,-0.541154,0.570982,2.022829,0.600011,1.231275,0.236739,0.338703,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709,1.247309,0.578961,0.551067,1.130912,5.200572,2.784500,0.707802,0.406606,0.555574,0.760566,...,-0.868539,-0.169519,0.305517,-0.631862,1.542295,0.729348,0.456026,0.707201,-0.044002,0.765652
710,0.181579,0.144406,0.233969,0.098897,0.635072,0.227591,0.180627,0.165571,0.220445,0.257921,...,-0.026890,-0.103319,0.137398,-0.024586,0.221549,0.115677,0.009149,0.103917,0.062139,0.091301
711,0.096385,0.093463,0.128921,0.107463,0.558667,0.091153,-0.036066,0.015429,0.108094,0.180066,...,-0.261751,-0.269424,0.055420,-0.174134,0.094984,0.000811,0.005686,0.149935,-0.050900,0.019170
712,0.024928,0.068197,-0.146074,-0.041722,0.451191,0.048453,-0.042733,0.084725,0.139246,0.164528,...,-0.284240,-0.155504,0.040756,-0.045641,0.113998,-0.036030,0.071052,0.067279,-0.112358,0.002270


In [30]:
result = result.iloc[:, :-1]

In [32]:
rf_model.fit(full_features2, full_labels2)
preds3= rf_model.predict(one_hot_test.toarray())
sample_columns = sample_submission.columns
sample_columns= sample_columns[1:]
submission_df3 = pd.DataFrame(preds3, columns=sample_columns)
submission_df3.insert(0, 'id', range(255))
no_public_data_rows3 = submission_df3[~submission_df3['id'].isin(cols_public)]
mse_ridge = mean_squared_error(no_public_data_rows3, no_provem_public_rows)
print(f'Error cuadrático medio en datos de validación (Ridge): {mse_ridge:.2f}')
public_data_rows3 = submission_df3[submission_df3['id'].isin(cols_public)]
mse_ridge = mean_squared_error(public_data_rows3, provem_public_rows)
print(f'Error cuadrático medio en datos de validación (Ridge): {mse_ridge:.2f}')

ValueError: Shape of passed values is (255, 18210), indices imply (255, 18211)

In [159]:
submission_df3

,id,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0,0.307006,0.175051,0.226601,0.723974,0.842409,0.521219,0.009332,0.379985,0.076783,...,-0.360935,-0.130687,-0.052597,0.215249,0.321890,0.514935,0.468769,0.422366,-0.476329,0.037028
1,1,-0.118997,0.035893,0.005977,0.077810,0.122944,0.022254,-0.052512,0.109032,0.053586,...,-0.062027,-0.024543,-0.079146,0.008270,0.046374,0.002716,0.085532,0.051431,-0.061012,-0.037401
2,2,2.696590,2.774901,2.195594,2.440472,2.865039,3.481642,2.442608,2.659288,2.425413,...,2.494336,2.296127,2.503573,2.648067,2.488647,2.699436,2.632042,2.498684,2.365986,2.310048
3,3,-0.006210,0.212920,0.178367,0.227740,0.301853,0.226250,0.054203,0.277345,0.183039,...,0.025160,0.089907,0.073190,0.211301,0.209420,0.172110,0.200757,0.182301,0.082749,0.064292
4,4,0.032053,0.023264,0.076018,0.171156,0.336488,0.251468,-0.039511,0.168894,0.005092,...,-0.197795,-0.046762,-0.091879,0.050717,0.096884,0.140898,0.133433,0.144320,-0.122749,-0.056040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,250,0.063643,0.049327,-0.019707,-0.034564,0.410426,0.039139,0.013941,0.081985,0.162132,...,-0.179258,-0.178602,0.037340,-0.047535,0.101239,0.013746,-0.024485,0.041659,-0.036802,-0.000453
251,251,0.063643,0.049327,-0.019707,-0.034564,0.410426,0.039139,0.013941,0.081985,0.162132,...,-0.179258,-0.178602,0.037340,-0.047535,0.101239,0.013746,-0.024485,0.041659,-0.036802,-0.000453
252,252,0.063643,0.049327,-0.019707,-0.034564,0.410426,0.039139,0.013941,0.081985,0.162132,...,-0.179258,-0.178602,0.037340,-0.047535,0.101239,0.013746,-0.024485,0.041659,-0.036802,-0.000453
253,253,-0.004717,-0.030581,-0.032890,-0.405425,0.235213,0.066662,0.232809,0.313503,0.976083,...,0.053125,0.102676,-0.249179,-0.114684,-0.099318,0.090518,-0.124596,0.058987,-0.032806,0.104820


In [160]:
provem

,id,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0,0.462067,0.333164,0.282393,0.864314,1.103537,0.681204,-0.011400,0.511585,0.091167,...,-0.487438,-0.076190,-0.074838,0.360439,0.399049,0.653428,0.619294,0.534062,-0.658728,0.041275
1,1,-0.077632,0.014408,0.065511,0.031914,0.278137,0.326295,-0.030011,0.139525,0.150199,...,0.013403,0.030177,-0.047023,0.109163,0.123197,0.029427,0.115830,0.036995,-0.050327,-0.047864
2,2,4.516325,4.453264,4.074874,4.208816,4.796946,5.500780,4.201675,4.470567,4.194056,...,4.245542,4.141732,4.264077,4.467162,4.335662,4.475067,4.417105,4.258112,4.102206,4.052730
3,3,0.061848,0.264810,0.280930,0.296526,0.416424,0.352968,0.115435,0.344529,0.244837,...,0.095571,0.175771,0.151149,0.319440,0.310137,0.279016,0.276906,0.243324,0.156719,0.114914
4,4,0.108198,0.015672,0.119478,0.222975,0.460281,0.378904,-0.039368,0.199365,-0.022555,...,-0.268710,-0.055975,-0.104511,0.077440,0.129164,0.191859,0.162400,0.181690,-0.155262,-0.069485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,250,0.078685,0.047404,-0.134128,-0.020459,0.282464,-0.153848,-0.008667,0.120142,0.155904,...,-0.185034,-0.209786,-0.009008,-0.028455,0.071896,-0.064984,0.035646,0.131742,-0.072857,-0.014499
251,251,0.192609,0.015485,0.267063,0.230512,1.300028,0.551453,0.099151,-0.022518,0.393088,...,-0.446241,-0.508816,0.125177,-0.094444,0.305428,0.084690,-0.031948,0.223898,0.033906,-0.009356
252,252,0.053263,0.148030,-0.022974,0.046427,0.364049,-0.052244,-0.044395,0.071997,0.107999,...,-0.241901,-0.188191,0.083745,-0.043139,0.120209,-0.040359,0.019710,0.183417,-0.014529,0.018427
253,253,1.072385,0.484715,-0.285435,0.361732,3.906288,2.971852,0.429157,0.514162,0.396571,...,-0.488719,0.278769,-0.455760,-0.515792,1.173396,0.572009,0.002834,0.209050,-0.308653,0.263124


In [170]:
#submission_df3 = submission_df3.iloc[:, 1:]
#submission_df3


submission_df3.to_csv("/home/roger/Baixades/provarandom.csv")

In [169]:
submission_df3.index.name = 'id'
submission_df3

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
id,,,,,,,,,,,,,,,,,,,,,
0,0.307006,0.175051,0.226601,0.723974,0.842409,0.521219,0.009332,0.379985,0.076783,0.542557,...,-0.360935,-0.130687,-0.052597,0.215249,0.321890,0.514935,0.468769,0.422366,-0.476329,0.037028
1,-0.118997,0.035893,0.005977,0.077810,0.122944,0.022254,-0.052512,0.109032,0.053586,0.116082,...,-0.062027,-0.024543,-0.079146,0.008270,0.046374,0.002716,0.085532,0.051431,-0.061012,-0.037401
2,2.696590,2.774901,2.195594,2.440472,2.865039,3.481642,2.442608,2.659288,2.425413,2.512917,...,2.494336,2.296127,2.503573,2.648067,2.488647,2.699436,2.632042,2.498684,2.365986,2.310048
3,-0.006210,0.212920,0.178367,0.227740,0.301853,0.226250,0.054203,0.277345,0.183039,0.243569,...,0.025160,0.089907,0.073190,0.211301,0.209420,0.172110,0.200757,0.182301,0.082749,0.064292
4,0.032053,0.023264,0.076018,0.171156,0.336488,0.251468,-0.039511,0.168894,0.005092,0.239106,...,-0.197795,-0.046762,-0.091879,0.050717,0.096884,0.140898,0.133433,0.144320,-0.122749,-0.056040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,0.063643,0.049327,-0.019707,-0.034564,0.410426,0.039139,0.013941,0.081985,0.162132,0.182707,...,-0.179258,-0.178602,0.037340,-0.047535,0.101239,0.013746,-0.024485,0.041659,-0.036802,-0.000453
251,0.063643,0.049327,-0.019707,-0.034564,0.410426,0.039139,0.013941,0.081985,0.162132,0.182707,...,-0.179258,-0.178602,0.037340,-0.047535,0.101239,0.013746,-0.024485,0.041659,-0.036802,-0.000453
252,0.063643,0.049327,-0.019707,-0.034564,0.410426,0.039139,0.013941,0.081985,0.162132,0.182707,...,-0.179258,-0.178602,0.037340,-0.047535,0.101239,0.013746,-0.024485,0.041659,-0.036802,-0.000453


In [174]:
provemmm = provem.set_index('id')


aaaa = provemmm*0,9 + submission_df3 * 0,1
aaaa

(     A1BG  A1BG-AS1  A2M  A2M-AS1  A2MP1  A4GALT  AAAS  AACS  AAGAB  AAK1  \
 id                                                                          
 0     0.0       0.0  0.0      0.0    0.0     0.0  -0.0   0.0    0.0   0.0   
 1    -0.0       0.0  0.0      0.0    0.0     0.0  -0.0   0.0    0.0   0.0   
 2     0.0       0.0  0.0      0.0    0.0     0.0   0.0   0.0    0.0   0.0   
 3     0.0       0.0  0.0      0.0    0.0     0.0   0.0   0.0    0.0   0.0   
 4     0.0       0.0  0.0      0.0    0.0     0.0  -0.0   0.0   -0.0   0.0   
 ..    ...       ...  ...      ...    ...     ...   ...   ...    ...   ...   
 250   0.0       0.0 -0.0     -0.0    0.0    -0.0  -0.0   0.0    0.0   0.0   
 251   0.0       0.0  0.0      0.0    0.0     0.0   0.0  -0.0    0.0   0.0   
 252   0.0       0.0 -0.0      0.0    0.0    -0.0  -0.0   0.0    0.0   0.0   
 253   0.0       0.0 -0.0      0.0    0.0     0.0   0.0   0.0    0.0   0.0   
 254   0.0      -0.0 -0.0     -0.0    0.0     0.0   0.0   0.0   

In [180]:
final= 0.95*provemmm + 0.05*submission_df3
final

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
id,,,,,,,,,,,,,,,,,,,,,
0,0.454314,0.325259,0.279603,0.857297,1.090481,0.673205,-0.010364,0.505005,0.090448,0.756958,...,-0.481113,-0.078915,-0.073726,0.353180,0.395191,0.646504,0.611768,0.528477,-0.649608,0.041063
1,-0.079700,0.015483,0.062535,0.034209,0.270377,0.311093,-0.031136,0.138000,0.145368,0.083891,...,0.009632,0.027441,-0.048629,0.104119,0.119356,0.028091,0.114315,0.037717,-0.050861,-0.047341
2,4.425338,4.369346,3.980910,4.120399,4.700351,5.399823,4.113721,4.380003,4.105624,4.180228,...,4.157982,4.049452,4.176052,4.376207,4.243311,4.386285,4.327852,4.170141,4.015395,3.965596
3,0.058446,0.262215,0.275801,0.293087,0.410696,0.346632,0.112373,0.341170,0.241748,0.303408,...,0.092051,0.171478,0.147251,0.314033,0.305101,0.273670,0.273098,0.240273,0.153021,0.112383
4,0.104391,0.016052,0.117305,0.220384,0.454092,0.372532,-0.039375,0.197842,-0.021173,0.294558,...,-0.265164,-0.055514,-0.103880,0.076104,0.127550,0.189311,0.160951,0.179821,-0.153636,-0.068812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,0.077933,0.047501,-0.128407,-0.021164,0.288862,-0.144198,-0.007536,0.118234,0.156215,0.208026,...,-0.184745,-0.208227,-0.006691,-0.029409,0.073363,-0.061047,0.032639,0.127238,-0.071054,-0.013797
251,0.186161,0.017177,0.252725,0.217258,1.255548,0.525837,0.094891,-0.017293,0.381540,0.218000,...,-0.432892,-0.492305,0.120785,-0.092099,0.295218,0.081142,-0.031575,0.214786,0.030371,-0.008910
252,0.053782,0.143095,-0.022811,0.042377,0.366368,-0.047675,-0.041478,0.072497,0.110706,0.220744,...,-0.238768,-0.187712,0.081425,-0.043359,0.119261,-0.037654,0.017501,0.176329,-0.015642,0.017483


In [181]:
final.to_csv("/home/roger/Baixades/final.csv")

In [34]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
#from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from py_boost.gpu import *
from py_boost.multioutput.sketching import *

/home/roger/anaconda3/lib/python3.9/site-packages/cupyx/scipy/sparse/_index.py:22: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  _bool_scalar_types = (bool, numpy.bool, numpy.bool_)
/home/roger/anaconda3/lib/python3.9/site-packages/cupyx/scipy/sparse/_index.py:22: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  _bool_scalar_types = (bool, numpy.bool, numpy.bool_)
/home/roger/anaconda3/lib/python3.9/site-packages/cupyx/scipy/sparse/_index.py:22: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  _bool_scalar_types = (bool, numpy.bool, numpy.bool_)
/home/roger/anaconda3/lib/python3.9/site-packages/cupyx/scipy/sparse/_index.py:22: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  _bool_scalar_types = (bool, numpy.bool, numpy.bool_)
/home/roger/anaconda3/lib/python3.9/site-packages/cupyx/scipy/sparse/_index.py:22: Futur